In [ ]:
# https://www.lazard.com/research-insights/2023-levelized-cost-of-energyplus/

In [ ]:
%pip install pyomo

In [ ]:
import pyomo.environ as pyo

battery capacity less than or equal to off peak requirement

stored power = peak_requirement - solar_capacity

off peak cost = off peak grid price *  (off peak requirement - solar capacity)

peak cost = peak requirement - solar capacity

minimize total cost

In [ ]:
model = pyo.ConcreteModel()
model.solar_capacity = pyo.Var(within=pyo.NonNegativeReals)
model.peak_grid_consumption = pyo.Var(within=pyo.NonNegativeReals)
model.off_peak_grid_usage = pyo.Var(within=pyo.NonNegativeReals)
model.peak_load = pyo.Param(initialize=10)
model.off_peak_load = pyo.Param(initialize=5)
model.battery_capacity = pyo.Var(within=pyo.NonNegativeReals)

model.peak_grid_usage = pyo.Var(within=pyo.NonNegativeReals)

model.minimizeCost = pyo.Objective(
    expr=model.peak_grid_usage + model.off_peak_grid_usage,
    sense=pyo.minimize,
)

model.abs_constraint1 = pyo.Constraint(
    expr=model.peak_load >= model.peak_grid_consumption - model.solar_capacity
)
model.abs_constraint2 = pyo.Constraint(
    expr=model.peak_load >= -(model.peak_grid_consumption - model.solar_capacity)
)

model.peak_constraint = pyo.Constraint(
    expr=model.peak_load <= model.peak_grid_consumption + model.solar_capacity
)

model.off_peak_constraint = pyo.Constraint(
    expr=model.off_peak_load <= model.off_peak_grid_usage + model.battery_capacity
)

optimizer = pyo.SolverFactory("glpk")
optimizer.solve(model)
print(model.display())